In [1]:
import pandas as pd
import numpy as np
import time
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

/home/varghese/miniconda3/envs/fuzzyname/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [2]:
#Load dataset
# filename = "../Nanonets_Fuzzymatching/datasets/companynames_dataset.csv"
# df = pd.read_csv(filename, sep=';', usecols=['name'])
# df.to_pickle("data.pickle") 

df = pd.read_pickle("data.pickle")
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040569 entries, 0 to 1040568
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   name    1040568 non-null  object
dtypes: object(1)
memory usage: 7.9+ MB
None
                         name
0                 balteau ndt
1   ex nihilo creations india
2  watsontown alliance church
3         uw zorgcompaan b.v.
4          dynamic life coach


In [3]:
#Data preprocessing
#Convert to lowercase
df["name"] = df["name"].str.lower()
#Remove punctuation
df["name"] = df["name"].str.replace('[^\w\s]','')
#Remove stopwords
stop = ["agency","gmbh","pa","and" , "group","pc","assn",\
        "hotel","pharmacy","assoc","hotels","plc","associates",\
        "inc","pllc","association","incorporated","restaurant",\
        "bank","international","sa","bv","intl","sales",\
        "co","limited","services","comp","company","llc","store",
        "ltd","travel","dmd","manufacturing","enterprises","mfg"]
df["name"] = df["name"]\
            .apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))
print(df.head())

/home/varghese/miniconda3/envs/fuzzyname/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


                         name
0                 balteau ndt
1   ex nihilo creations india
2  watsontown alliance church
3              uw zorgcompaan
4          dynamic life coach


In [4]:
#Compute Vectors
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tf_idf_matrix = tfidf_vectorizer.fit_transform(df['name'].values.astype('U'))
print(tf_idf_matrix[0])

  (0, 143395)	0.6191461163997317
  (0, 1009238)	0.4830281286780922
  (0, 143394)	0.6191461163997317


In [7]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [13]:
start_time = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.6)
print("Total computation time:",time.time()-start_time)

#Save for later use
# filehandler = open("matches.pkl", 'wb') 
# pickle.dump(matches, filehandler)

# filehandler = open("matches.pkl","rb")
# matches = pickle.load(filehandler)


Total computation time: 231.66275358200073
